# Image Recognition Using Transfer Learning

### Mounts drive on google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Import all necessary flies

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
#from keras import optimizers

### Verify TensorFlow version

In [ ]:
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
print("GPU : ",tf.test.gpu_device_name())

### Load the data
we'll  need to make sure the input data is resized to 224x224 or 229x229 pixels as required by the networks.



In [ ]:
zip_file="/content/drive/My Drive/Celebs_Dataset/"

### Prepare training and validation  dataset
Create the training and validation directories 

In [ ]:
data_dir = os.path.join(os.path.dirname(zip_file), 'data')
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')

In [ ]:
print(train_dir)

In [ ]:
import time
import os
from os.path import exists

def count(dir, counter=0):
    "returns number of files in dir and subdirs"
    for pack in os.walk(dir):
        for f in pack[2]:
            counter += 1
    return dir + " : " + str(counter) + " files"

In [ ]:
print('total images for training :', count(train_dir))
print('total images for validation :', count(validation_dir))

### Label mapping

You'll also need to load in a mapping from category label to category name. 

In [ ]:
classes = ["Elon Musk", "Bill Gates", "Steve Jobs", "Sundar Pichai", "Jeff Beroz", "Jack Ma", 
           "Satya Nadela", "Larry Page", "Mark Zukerberg", "Arvind Krishna"]
    
print (classes)

In [ ]:
print('Number of classes:',len(classes))

### Select the Pre-trained Model to use

In [ ]:
module_selection = ("mobilenet_v2", 224, 1280)

handle_base, pixels, FV_SIZE = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/2".format(handle_base)
IMAGE_SIZE = (pixels, pixels)

print("Using {} with input size {} and output dimension {}".format(
  MODULE_HANDLE, IMAGE_SIZE, FV_SIZE))

BATCH_SIZE = 64 

### Data Preprocessing

##### Let's set up data generators that will read pictures in our source folders, and perform preprocessing (Data augmentation). Inputs are suitably resized for the selected module. Dataset augmentation (i.e., random distortions of an image each time it is read) improves training, esp. when fine-tuning.


In [ ]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, 
    shuffle=False, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)

do_data_augmentation = True
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, 
      height_shift_range=0.2,
      shear_range=0.2, 
      zoom_range=0.2,
      fill_mode='nearest' )
else:
  train_datagen = validation_datagen
  
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    subset="training", 
    shuffle=True, 
    seed=42,
    color_mode="rgb", 
    class_mode="categorical",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE)


### Build the model
##### All it takes is to put a linear classifier on top of the feature_extractor_layer with the Hub module.

##### For speed, we start out with a non-trainable feature_extractor_layer, but you can also enable fine-tuning for greater accuracy.

In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE,
                                   input_shape=IMAGE_SIZE+(3,),
                                   output_shape=[FV_SIZE])

In [ ]:
from keras.applications import MobileNet

img_rows, img_cols = 224, 224 

### Re-loads the MobileNet model without the top or FC layers

In [ ]:
MobileNet = MobileNet(weights = 'imagenet', 
                 include_top = False, 
                 input_shape = (img_rows, img_cols, 3))

### For fine tuning

In [ ]:
do_fine_tuning = True
base_model = MobileNet
if do_fine_tuning:
    feature_extractor.trainable = True
    for layer in base_model.layers[-30:]:
        layer.trainable = False
else:
    feature_extractor.trainable = False    

### For building module

In [ ]:
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                           kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])

model.summary()

### Compiling Model
##### Specify Loss Function and Optimizer

In [ ]:
LEARNING_RATE = 0.001

model.compile(
   optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE), 
   loss='categorical_crossentropy',
   metrics=['accuracy'])

### Train model using validation dataset for validate each steps

In [ ]:
EPOCHS=25 #@param {type:"integer"}

history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples//train_generator.batch_size,
        epochs=EPOCHS,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples//validation_generator.batch_size)

### Random test
##### Random sample images from validation dataset and predict

In [ ]:
import cv2
import itertools
import random
from collections import Counter
from glob import iglob

##### To load random image

In [ ]:
def load_image(filename):
    img = cv2.imread(os.path.join(data_dir, validation_dir, filename))
    img = cv2.resize(img, (IMAGE_SIZE[0], IMAGE_SIZE[1]) )
    img = img /255
    
    return img

##### To predict image

In [ ]:
def predict(image):
    probabilities = model.predict(np.asarray([img]))[0]
    class_idx = np.argmax(probabilities)
    
    return {classes[class_idx]: probabilities[class_idx]}

##### Shows prediction and image

In [ ]:
for idx, filename in enumerate(random.sample(validation_generator.filenames, 5)):
    print("SOURCE: class: %s, file: %s" % (os.path.split(filename)[0], filename))
    
    img = load_image(filename)
    prediction = predict(img)
    print("PREDICTED: class: %s, confidence: %f" % (list(prediction.keys())[0], list(prediction.values())[0]))
    plt.imshow(img)
    plt.figure(idx)    
    plt.show()

## Save the Model
##### Now that we have trained the model, export it as a saved model

In [ ]:
import time
t = time.time()

export_path = "/content/drive/My Drive/Face Recognition /Model/{}".format("face_reco")
tf.keras.experimental.export_saved_model(model, export_path)
